## 1. Import Libraries 

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer 
from sklearn.metrics import silhouette_score , silhouette_samples, calinski_harabasz_score
from sklearn.mixture import GaussianMixture
import plotly.express as px
import warnings
import joblib
warnings.filterwarnings('ignore')

# Set random seed untuk reprodusibilitas
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Load Dataset

Dataset diambil dari GitHub repository sesuai ketentuan kompetisi.

In [88]:
# Load dataset
url = 'https://raw.githubusercontent.com/micelll/SPARC-2026/main/SPARC_dataset.csv'
df = pd.read_csv(url)
df.head()

,Customer ID,Kelurahan,Kecamatan,Kode POS,Cash/Credit,Kode Dealer,Finance Company,Tenor,Gender,Tgl Lahir,...,range dp,wilayah,9 segment,kode motor,OTR,tahun rakit,DLR group,tgl cetak,tgl mohon,Kode Kota-Provinsi
0,CUST-159769,Baru Ilir,Balikpapan Barat,76131,2.0,12756.0,4,3,2,1992-03-03 0:00:00,...,2 - 3 juta,6471,AT LOW,HN,18685000.0,NaN,NaN,NaN,02-01-2019,6471-6400
1,CUST-164551,BARU TENGAH,BALIKPAPAN BARAT,76132,2.0,733.0,1,1,2,1991-04-27 0:00:00,...,3 jt up,6471,AT LOW,HN,18685000.0,NaN,NaN,NaN,02012019,6471-6400
2,CUST-159733,BARU TENGAH,BALIKPAPAN BARAT,76132,1.0,733.0,N,N,2,1982-01-04 0:00:00,...,kurang 1 juta,6471,AT MID,HR,20775000.0,NaN,NaN,NaN,02012019,6471-6400
3,CUST-164422,BARU TENGAH,BALIKPAPAN BARAT,76132,1.0,733.0,N,N,2,1997-06-26 0:00:00,...,krg 1 jt,6471,AT MID,HR,20775000.0,NaN,NaN,NaN,02012019,6471-6400
4,CUST-155472,Baru Ulu,Balikpapan Barat,76133,2.0,11160.0,1,2,1,1996-07-12 0:00:00,...,2 - 3 jt,6471,AT MID,HR,20775000.0,NaN,NaN,NaN,02-01-2019,6471-6400


a

In [89]:
# Informasi umum dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319978 entries, 0 to 319977
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Customer ID         319964 non-null  object 
 1   Kelurahan           319962 non-null  object 
 2   Kecamatan           319963 non-null  object 
 3   Kode POS            319963 non-null  object 
 4   Cash/Credit         319963 non-null  float64
 5   Kode Dealer         319963 non-null  float64
 6   Finance Company     319889 non-null  object 
 7   Tenor               319942 non-null  object 
 8   Gender              319962 non-null  object 
 9   Tgl Lahir           319963 non-null  object 
 10  Agama               319963 non-null  object 
 11  Pekerjaan           319963 non-null  object 
 12  umur                319963 non-null  float64
 13  dp aktual           319234 non-null  object 
 14  cicilan             319276 non-null  object 
 15  warna               319963 non-nul

In [90]:
df['Cash/Credit'] = df['Cash/Credit'].astype('Int64')
df['Kode Dealer'] = df['Kode Dealer'].astype('Int64')
df['Finance Company'] = df['Finance Company'].replace('N', np.nan).astype('Int64')
df['Tenor'] = df['Tenor'].replace('N', np.nan).astype('Int64')
df['Gender'] = df['Gender'].replace('N', np.nan).astype('Int64')
df['Agama'] = df['Agama'].replace('N', np.nan).astype('Int64')

In [91]:
df.isnull().sum()
#cek persentassi mising value
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_percentage

Customer ID            0.004375
Kelurahan              0.005000
Kecamatan              0.004688
Kode POS               0.004688
Cash/Credit            0.004688
Kode Dealer            0.004688
Finance Company       43.591434
Tenor                 43.582996
Gender                 3.146779
Tgl Lahir              0.004688
Agama                  2.569552
Pekerjaan              0.004688
umur                   0.004688
dp aktual              0.232516
cicilan                0.219390
warna                  0.004688
dealer                 0.004688
type series            0.004688
range dp               0.004688
wilayah                0.004688
9 segment              0.004688
kode motor             0.016876
OTR                    0.005938
tahun rakit            3.097713
DLR group             80.687110
tgl cetak             95.866591
tgl mohon              0.004375
Kode Kota-Provinsi     0.000000
dtype: float64

In [92]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

In [93]:
# Standardisasi kategori
df['range dp'] = df['range dp'].str.upper().str.strip().replace({
    '2 - 3 JT': '2-3JT',
    '2 - 3 JUTA': '2-3JT',
    '3 JUTA UP': '>3JT',
    '3 JT UP': '>3JT',
    'KURANG 1 JUTA': '<1JT',
    'KRG 1 JUTA': '<1JT',
    'KRG 1 JT': '<1JT',
    '1 - 2 JT': '1-2JT',
    '1 - 2 JUTA': '1-2JT',
    'KURANG 1 JT': '<1JT'
})
#drop baris yang memiliki nilai range dp yang nan
df = df.dropna(subset=['range dp'])

print(f"Range DP unique: {df['range dp'].unique()}")

Range DP unique: ['2-3JT' '>3JT' '<1JT' '1-2JT']


In [94]:
#standardisasi kategori lainnya
df['Agama'] = df['Agama'].astype(str).str.upper().str.strip()
df['Pekerjaan'] = df['Pekerjaan'].astype(str).str.upper().str.strip()
df['Gender'] = df['Gender'].astype(str).str.upper().str.strip()
df['warna'] = df['warna'].str.upper().str.strip()

In [95]:
# df['tgl mohon'] = pd.to_datetime(df['tgl mohon'], errors='coerce', dayfirst=True)
# df['tgl cetak'] = pd.to_datetime(df['tgl cetak'], errors='coerce', dayfirst=True)
# df['Tgl Lahir'] = pd.to_datetime(df['Tgl Lahir'], errors='coerce', dayfirst=True)

# print(f"tgl mohon: {df['tgl mohon'].dtype}")
# print(f"tgl cetak: {df['tgl cetak'].dtype}")
# print(f"Tgl Lahir: {df['Tgl Lahir'].dtype}")
# print(df['tgl mohon'].head())
# print(df['tgl cetak'].head())
# print(df['Tgl Lahir'].head())


Labeling

In [96]:
df_labeling = df.copy()

In [97]:
df_labeling = df_labeling[['Tenor', 'tahun rakit', 'Cash/Credit', 'Pekerjaan']]
df_labeling.head()

,Tenor,tahun rakit,Cash/Credit,Pekerjaan
0,3,NaN,2,2E
1,1,NaN,2,2B
2,<NA>,NaN,1,4B
3,<NA>,NaN,1,11
4,2,NaN,2,2E


In [98]:
df_labeling['Tenor'] = pd.to_numeric(df_labeling['Tenor'], errors='coerce')
df_labeling['tahun rakit'] = pd.to_numeric(df_labeling['tahun rakit'], errors='coerce')
df_labeling['Cash/Credit'] = pd.to_numeric(df_labeling['Cash/Credit'], errors='coerce')
df_labeling['Pekerjaan'] = df_labeling['Pekerjaan'].astype(str).str.upper()

df_labeling.dropna(inplace=True)

# SETELAH dropna baru convert ke Int64
df_labeling['Tenor'] = df_labeling['Tenor'].astype('Int64')
df_labeling['tahun rakit'] = df_labeling['tahun rakit'].astype('Int64')
df_labeling['Cash/Credit'] = df_labeling['Cash/Credit'].astype('Int64')

In [99]:
df_labeling.head()

,Tenor,tahun rakit,Cash/Credit,Pekerjaan
9882,1,2019,2,7
9883,2,2019,2,2D
9884,2,2019,2,5
9885,2,2019,2,8
9886,1,2019,2,8


In [100]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174548 entries, 9882 to 319976
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Tenor        174548 non-null  Int64 
 1   tahun rakit  174548 non-null  Int64 
 2   Cash/Credit  174548 non-null  Int64 
 3   Pekerjaan    174548 non-null  object
dtypes: Int64(3), object(1)
memory usage: 7.2+ MB


In [101]:
df_labeling.nunique()

Tenor           5
tahun rakit    10
Cash/Credit     2
Pekerjaan      28
dtype: int64

In [102]:
categorical_cols = ['Tenor', 'tahun rakit', 'Cash/Credit', 'Pekerjaan']

for col in categorical_cols:
    if col in df_labeling.columns:
        print(f"\n{col}: {df_labeling[col].nunique()} unique")
        print(df_labeling[col].unique()[:10])


Tenor: 5 unique
<IntegerArray>
[1, 2, 3, 4, 6]
Length: 5, dtype: Int64

tahun rakit: 10 unique
<IntegerArray>
[2019, 2018, 2017, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
Length: 10, dtype: Int64

Cash/Credit: 2 unique
<IntegerArray>
[2, 1]
Length: 2, dtype: Int64

Pekerjaan: 28 unique
['7' '2D' '5' '8' '16' '2E' '4D' '2B' '4E' '4A']


tes

In [103]:
def calculate_priority_score(row):
    score = 0
    
    # 1. Tenor (bobot 40)
    if row['Tenor'] == 1:
        score += 40
    elif row['Tenor'] == 2:
        score += 20
    elif row['Tenor'] == 3:
        score += 10
    
    # 2. Vehicle Age (bobot 30)
    vehicle_age = 2026 - row['tahun rakit']
    if 3 <= vehicle_age <= 5:
        score += 30
    elif 6 <= vehicle_age <= 8:
        score += 20
    elif vehicle_age > 8:
        score += 10
    
    # 3. Cash/Credit (bobot 15)
    if row['Cash/Credit'] == 1:
        score += 15
    
    # 4. Pekerjaan (bobot 15)
    high_value_jobs = ['1', '6', '7', '10', '12', '13', '2D', '4D']
    if row['Pekerjaan'] in high_value_jobs:
        score += 15
    
    return score

df_labeling['score'] = df_labeling.apply(calculate_priority_score, axis=1)
df_labeling['priority_label'] = df_labeling['score'].apply(lambda x: 'High' if x >= 70 else ('Medium' if x >= 40 else 'Low'))

print(df_labeling['priority_label'].value_counts())

priority_label
Medium    97557
Low       55952
High      21039
Name: count, dtype: int64


In [104]:
df_labeling.head()

,Tenor,tahun rakit,Cash/Credit,Pekerjaan,score,priority_label
9882,1,2019,2,7,75,High
9883,2,2019,2,2D,55,Medium
9884,2,2019,2,5,40,Medium
9885,2,2019,2,8,40,Medium
9886,1,2019,2,8,60,Medium


merge

In [105]:
df = df.merge(df_labeling[['priority_label', 'score']], left_index=True, right_index=True, how='left')
df.head()

,Customer ID,Kelurahan,Kecamatan,Kode POS,Cash/Credit,Kode Dealer,Finance Company,Tenor,Gender,Tgl Lahir,...,9 segment,kode motor,OTR,tahun rakit,DLR group,tgl cetak,tgl mohon,Kode Kota-Provinsi,priority_label,score
0,CUST-159769,Baru Ilir,Balikpapan Barat,76131,2,12756,4,3,2,1992-03-03 0:00:00,...,AT LOW,HN,18685000.0,NaN,NaN,NaN,02-01-2019,6471-6400,NaN,NaN
1,CUST-164551,BARU TENGAH,BALIKPAPAN BARAT,76132,2,733,1,1,2,1991-04-27 0:00:00,...,AT LOW,HN,18685000.0,NaN,NaN,NaN,02012019,6471-6400,NaN,NaN
2,CUST-159733,BARU TENGAH,BALIKPAPAN BARAT,76132,1,733,<NA>,<NA>,2,1982-01-04 0:00:00,...,AT MID,HR,20775000.0,NaN,NaN,NaN,02012019,6471-6400,NaN,NaN
3,CUST-164422,BARU TENGAH,BALIKPAPAN BARAT,76132,1,733,<NA>,<NA>,2,1997-06-26 0:00:00,...,AT MID,HR,20775000.0,NaN,NaN,NaN,02012019,6471-6400,NaN,NaN
4,CUST-155472,Baru Ulu,Balikpapan Barat,76133,2,11160,1,2,1,1996-07-12 0:00:00,...,AT MID,HR,20775000.0,NaN,NaN,NaN,02-01-2019,6471-6400,NaN,NaN
